# Transfer Learning

**1. What is Transfer Learning ?**

*Ans :* `Is is the method of using the already trained model on the dataset similar to the trained model data.`

In [1]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
from tensorflow.keras import initializers
from tensorflow.python.keras import activations

print(tf.__version__)

# downloading fashion_mnist data
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images = train_images / 255.0

test_images = test_images / 255.0       

activation = tf.keras.activations.tanh

c:\users\sonu.ramkumar.jha\desktop\experiments\env\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\users\sonu.ramkumar.jha\desktop\experiments\env\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\users\sonu.ramkumar.jha\desktop\experiments\env\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


2.5.0


In [2]:
model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(28, 28)),
tf.keras.layers.Dense(128, activation=activation),
tf.keras.layers.Dense(10)
])

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

# model summary
model.summary()



model.fit(train_images, train_labels, epochs=10)

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('test_loss', test_loss)
print('test_accuracy', test_acc)

model.save('model.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4795 - accuracy: 0.8278
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3686 - accuracy: 0.8658
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3341 - accuracy: 0.8774
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss

## Predicting image is coat or not


Since this model is already trained to classify 10 classes, therefore we don't need to train a model again from scratch 
to predict an image being `coat` or not. Insted we do some modification in the last layer of this model and used previous trained weight and bias, which is good to classify 10 classes.

In [3]:
# loading trained model

pretrained_fmnist_model = tf.keras.models.load_model('model.h5')
pretrained_fmnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [4]:
# checking the trainaible layers
for layer in pretrained_fmnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: True
dense: True
dense_1: True


In [5]:
# We don't trained the parameters before last layer 
for layer in pretrained_fmnist_model.layers[:-1]:
  layer.trainable = False

# checking the trainaible layer
for layer in pretrained_fmnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
dense_1: True


**Defining new `train label` and `test label`**

In [6]:

# index of 'coat' in train_labels is 4 

coat_train_lbl = train_labels==4
coat_train_lbl

array([False, False, False, ..., False, False, False])

In [7]:
coat_test_lbl = test_labels == 4
coat_test_lbl

array([False, False, False, ..., False, False, False])

In [8]:
# 
lower_pretrained_layers = pretrained_fmnist_model.layers[:-1]

# 
new_model = tf.keras.models.Sequential(lower_pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(1, activation='sigmoid')
)

In [9]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 100,609
Trainable params: 129
Non-trainable params: 100,480
_________________________________________________________________


In [10]:
loss = tf.keras.losses.binary_crossentropy
new_model.compile(loss=loss,
                  optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3),
                  metrics = ['accuracy'])

In [11]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 100,609
Trainable params: 129
Non-trainable params: 100,480
_________________________________________________________________


In [ ]:
history = new_model.fit(train_images, coat_train_lbl, epochs=10,
                    validation_data=(test_images, coat_test_lbl), verbose=2)

Epoch 1/10
1875/1875 - 4s - loss: 0.2915 - accuracy: 0.8820 - val_loss: 0.2225 - val_accuracy: 0.8982
Epoch 2/10
1875/1875 - 3s - loss: 0.2027 - accuracy: 0.9114 - val_loss: 0.1868 - val_accuracy: 0.9229
Epoch 3/10
1875/1875 - 3s - loss: 0.1747 - accuracy: 0.9294 - val_loss: 0.1683 - val_accuracy: 0.9331
Epoch 4/10
1875/1875 - 3s - loss: 0.1581 - accuracy: 0.9387 - val_loss: 0.1566 - val_accuracy: 0.9388
Epoch 5/10


## Observations : 
- As you can see the model starts training from the  val_loss 0.208, and val_accuracy  0.9210. Which is just after the trained model.
- 